# Question & Answering With AI

### 1. Basic Question & Answering

Let's first install the required libraries and load up our packages

In [17]:
pip install --upgrade langchain

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openai

  Using cached openai-1.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
  Using cached httpx-0.25.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Using cached openai-1.5.0-py3-none-any.whl (223 kB)
Using cached httpx-0.25.2-py3-none-any.whl (74 kB)
Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install beautifulsoup4

  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [48]:
pip install tiktoken

  Using cached tiktoken-0.5.2-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata (41 kB)
Using cached tiktoken-0.5.2-cp311-cp311-win_amd64.whl (786 kB)
Using cached regex-2023.10.3-cp311-cp311-win_amd64.whl (269 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key=os.environ['OPENAI_API_KEY']
base_url=os.environ['OPENAI_BASE_URL']

print(base_url)

https://devsquad-eastus-2.openai.azure.com/


## Summaries of Short Text

For summaries of short texts, the method is straightforward, in fact you don't need to do anything fancy other than simple prompting with instructions

In [3]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain import PromptTemplate

llm = AzureChatOpenAI(
    api_key=api_key,
    azure_endpoint=base_url, 
    api_version="2023-07-01-preview",
    model="gpt-4",
    temperature=0
)

template = """
Please provide a summary of the following text:

{text}
"""

prompt = PromptTemplate(input_variables=["text"], template=template)

let's test it with a small story:

In [4]:
story = """
Once upon a time, in a small village nestled between the mountains and the sea, lived a mighty dragon and a wise princess. The dragon, known for its fiery breath, was feared by all. The princess, on the other hand, was loved for her kindness and wisdom.
The princess was deeply concerned about the changing climate. She noticed the winters becoming harsher, the summers hotter, and the crops failing. She realized that the dragon’s fire, used to keep the villagers warm during the cold winters, was contributing to the rising temperatures.
She decided to have a conversation with the dragon. “Dear friend,” she began, “Our village is suffering because of the changing climate. The heat from your fire is making the summers unbearable and affecting our crops. We need to find a solution.”
The dragon, who cared for the village as much as the princess, agreed. They decided to limit the use of the dragon’s fire to only the coldest days of winter. The dragon also helped the villagers build energy-efficient homes to stay warm.
The princess didn’t stop there. She educated the villagers about the importance of sustainable living. They started planting more trees, recycling, and using renewable energy sources.
Over time, the village became a model of sustainability. The dragon and the princess showed everyone that with understanding, cooperation, and sustainable practices, it’s possible to combat climate change. And so, they continued to live in harmony with nature, proving that even in a story of a princess and a dragon, there’s room for real-world issues like climate change.
"""

In [5]:
final_prompt = prompt.format(text=story)
print(final_prompt)


Please provide a summary of the following text:


Once upon a time, in a small village nestled between the mountains and the sea, lived a mighty dragon and a wise princess. The dragon, known for its fiery breath, was feared by all. The princess, on the other hand, was loved for her kindness and wisdom.
The princess was deeply concerned about the changing climate. She noticed the winters becoming harsher, the summers hotter, and the crops failing. She realized that the dragon’s fire, used to keep the villagers warm during the cold winters, was contributing to the rising temperatures.
She decided to have a conversation with the dragon. “Dear friend,” she began, “Our village is suffering because of the changing climate. The heat from your fire is making the summers unbearable and affecting our crops. We need to find a solution.”
The dragon, who cared for the village as much as the princess, agreed. They decided to limit the use of the dragon’s fire to only the coldest days of winter. The

Finally let's use our LLM

In [6]:
message = HumanMessage(content=final_prompt)
output = llm([message])
print(output.content)

In a small village, a mighty dragon and a wise princess lived among the residents. The princess noticed the adverse effects of climate change, such as extreme weather and failing crops, and realized the dragon's fire was contributing to the problem. She approached the dragon, and together they agreed to limit the use of its fire to the coldest days and to help villagers build energy-efficient homes. The princess also led the village in adopting sustainable practices, such as planting trees, recycling, and using renewable energy. Over time, the village became a model of sustainability, demonstrating that cooperation and sustainable living can address climate change, with the dragon and princess living in harmony with nature and highlighting the relevance of environmental issues in their story.


This method works fine, but for longer text, it can become a pain to manage and you'll run into token limits. Luckily LangChain has out of the box support for different methods to summarize via their `load_summarize_chain`

## Summaries of Longer Text

Note: This method will also work for short text.

#### Load your data

We need to define a function to extract the article part of it using `BeautifulSoup4`

In [7]:
from bs4 import BeautifulSoup


def extract_article(content: BeautifulSoup) -> str:
    # Find all 'article' elements in the BeautifulSoup object
    article_elements = content.find_all("article")

    return str(article_elements.get_text())

Then we'll load our PDF text into a variable. The `WebBaseLoader` will convert our text for us

In [26]:
loader = WebBaseLoader("https://www.newyorker.com/news/letter-from-the-uk/the-collateral-damage-of-queen-elizabeths-glorious-reign")
data = loader.load()

Let's check how many tokens this would use:

In [27]:
num_tokens = llm.get_num_tokens(data[0].page_content)
print(f"The number of tokens in the message is {num_tokens}.")

The number of tokens in the message is 4769.


Let's see how many documents and characters we have in our data

In [28]:
print(f'You have {len(data)} documents in your data')
print(f'There are {len(data[0].page_content)} characters in your first document')

You have 1 documents in your data
There are 21761 characters in your first document


#### Chunk your data up into smaller documents
Let's split into smaller chunks and assume is too big.

In [29]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=500)
docs = text_splitter.split_documents(data)

In [30]:
print(f'You have {len(docs)} documents in your data')

You have 3 documents in your data


In [31]:
print(docs[0].page_content[:500])

The Collateral Damage of Queen Elizabeth’s Glorious Reign | The New YorkerSkip to main contentNewsletterStory SavedTo revisit this article, select My Account, then View saved storiesClose AlertSign InSearchSearchThe Latest2023 in ReviewNewsBooks & CultureFiction & PoetryHumor & CartoonsMagazinePuzzles & GamesVideoPodcastsGoings OnShopOpen Navigation MenuMenuStory SavedFind anything you save across the site in your account Close AlertLetter from the U.K.The Collateral Damage of Queen Elizabeth’s 


Now let's use the `load_summarize_chain` with `map_reduce` to summarize our text

In [32]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm=llm, chain_type="map_reduce")

In [33]:
output = chain.run(docs)

In [34]:
print(output)

Sam Knight's article in The New Yorker, based on insights from Tina Brown's "The Palace Papers," examines the personal challenges faced by the British Royal Family during Queen Elizabeth II's reign. The monarchy is depicted as a system where the Queen holds authority, while other royals are confined to ceremonial roles, leading to a sense of purposelessness and personal struggles. The Queen's children and grandchildren have had varied experiences, with some facing scandals and others seeking a life away from royal duties. The article touches on the future of the monarchy, highlighting Prince Charles's environmental activism and the potential stability offered by Prince William and Catherine's family. It also addresses broader issues such as calls for reparations and the possibility of countries distancing themselves from the monarchy. The summary concludes by mentioning The New Yorker's diverse content and Sam Knight's credentials as a staff writer and author.


: 